In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri


In [ ]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [ ]:
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'databricks/dolly-v2-3b',
	'SM_NUM_GPUS': json.dumps(1)
}

In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )

In [ ]:
class journallingClass:
    def __init__(self, journalType, title, category, dateCreated, mood, journalClass):
        self.journalType = journalType
        self.journalTitle = title
        self.journalCategory = category 
        self.dateCreated = dateCreated
        self.mood = mood
        self.journalClass = journalClass
    
    def getJournalClass(self):
        return self.journalClass
        
        
class journalEntryClass:
    def __init__(self, userEntry, harmIntent = 0, modelResponse = []):
        self.userEntry = userEntry 
        self.harmIntent = harmIntent
        self.modelResponse = modelResponse
#         self.safetyResponse = safetyResponse
    def printEntry(self):
        print(f"entry: {self.userEntry}, harmful intent: {self.harmIntent}, model response: {self.modelResponse}")

        
    def updateHarmIntent(self, harmIntent):
        self.harmIntent = harmIntent
    
    def addModelResponse(self, response):
        self.modelResponse.append(response)
        
class casualJournal(journallingClass):
    
    def __init__(self, entries = []):
        self.entries = []
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.entries.append(journalEntryClass(userEntry, harmIntent, modelResponse))
        return self.entries[-1]
        
    def printJournal(self):
        for entry in self.entries:
            entry.printEntry()
    
    
        
    
class catharticJournal(journallingClass):
    def __init__(self, stage = 0, entries = {}):
        self.stage = stage # default stage 0
        self.entries = entries
        
    
    def createStage(self, journalEntry):
        self.entries[f"stage{self.stage}"] = journalEntry
    
    def updateStage(self):
        if self.stage >=5:
            return 
        self.stage += 1
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.updateStage()
        self.entries[f"stage{self.stage}"] = journalEntryClass(userEntry, harmIntent, modelResponse)
        return self.entries[f"stage{self.stage}"]
    
    def printJournal(self):
        for stage in self.entries.keys():
            print(stage)
            self.entries[stage].printEntry()


In [ ]:
def generate_text(prompt):
    return predictor.predict({
	"inputs": prompt,
    })

def checkHarmfulIntent(journalEntry):
    # implement later
    journalEntry.updateHarmIntent(0)
    # If this is harmful update the harmful intent to be set to 1
    return False 

def buildHarmresponse():
    response = "As an AI language model, I don't have feelings or emotions, but I am designed to provide helpful and supportive responses to you. However, it seems like you are going through a challenging time and dealing with issues that may benefit from professional help, I would recommend that you consider speaking with a licensed therapist or counselor."
    return response

def generateCatharticResponse(journalEntry, journal):
    if journal.getJournalClass().stage == 5:
        prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. Can you give me other ways of thinking about the situation? This is the given scenario: {journalEntry.userEntry}"
    else:
        prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: {journalEntry.userEntry}"
    
    print(f"Prompt: {prompt}")
    response = generate_text(prompt)
    return response[0]

def generateCasualResponse(journalEntry):
    prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: {journalEntry.userEntry}"
    print(f"Prompt: {prompt}")
    res = generate_text(prompt)
    return res[0]

def promptSafetyDetector(journalEntry, safety_counter):
    return True

def generatePrompt(typeJ, journalEntry, journal):
    safety_counter = 0
    safety_threshold = 5
    
    while(safety_counter<safety_threshold):
        if typeJ == 1:
            response = generateCatharticResponse(journalEntry, journal)
        else:
            response = generateCasualResponse(journalEntry)
        # check safety
        safe = promptSafetyDetector(journalEntry, safety_counter)
        if safe:
            break
        safety_counter += 1
        
    if safety_counter == safety_threshold:
        response = "<default response - model response was unsafe.>"
    
    # add model response to current entry object
    journalEntry.addModelResponse(response)
    
    return response

In [ ]:
def main(response):
    
    if response["type"] == 0: # get type check from backend
        journalChild = casualJournal()
    else:
        journalChild = catharticJournal()
        
    # pull journalType, title, category, dateCreated from backend
    journal = journallingClass(response["type"], response["title"], response["category"], response["date"], response["mood"], journalChild)
    
    # pull journal entries, loop through and load 
    for entry, harmfulIntent, modelResponse in zip(response["entry"], response["harmfulIntent"], response["modelResponse"]):
        journal.getJournalClass().createNewEntry(entry, harmfulIntent, modelResponse)
    
    # get latest entry
    currentEntry = journal.getJournalClass().createNewEntry(response["entry"][-1])
    
    currentEntry.printEntry()
    
    
    harmIntent = checkHarmfulIntent(currentEntry)
    
    if harmIntent: 
        modelResponse = buildHarmresponse() 
        currentEntry.addModelResponse(modelResponse)
        result["modelResponse"].append(modelResponse)
        result["harmfulIntent"].append(1) # confirmm if 0/1 or true/false
        return result
    
    modelResponse = generatePrompt(response["type"], currentEntry, journal)
    
#     currentEntry.printEntry()
    journal.getJournalClass().printJournal()
    
    
    result = response
    result["modelResponse"].append(modelResponse)
    result["harmfulIntent"].append(0)
    return result
    